In [1]:
%load_ext autoreload  
%autoreload 2  
!hostname  
!pwd  
import os
import sys

print(sys.executable)
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
os.environ['LD_LIBRARY_PATH'] = '/data/vision/phillipi/akumar01/.mujoco/mujoco210/bin'

oliva-titanrtx-2.csail.mit.edu
/data/vision/phillipi/akumar01/synthetic-mdps/src
/data/vision/phillipi/akumar01/.virtualenvs/dt/bin/python


In [2]:
import os, sys, glob, pickle
from functools import partial  

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from einops import rearrange, reduce, repeat

from dataclasses import dataclass
import math

/data/vision/phillipi/akumar01/.virtualenvs/dt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
from torch import nn
from torch.nn import functional as F

In [4]:
class LayerNorm(nn.Module):
    """ LayerNorm but with an optional bias. PyTorch doesn't support simply bias=False """

    def __init__(self, ndim, bias):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None

    def forward(self, input):
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)

class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        # regularization
        self.attn_dropout = nn.Dropout(config.dropout)
        self.resid_dropout = nn.Dropout(config.dropout)
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.dropout = config.dropout
        # flash attention make GPU go brrrrr but support is only in PyTorch >= 2.0
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
        if not self.flash:
            print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
            # causal mask to ensure that attention is only applied to the left in the input sequence
            self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                        .view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k, v  = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        if self.flash:
            # efficient attention using Flash Attention CUDA kernels
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
        else:
            # manual implementation of attention
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
            att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
            att = F.softmax(att, dim=-1)
            att = self.attn_dropout(att)
            y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y

class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)
        self.gelu    = nn.GELU()
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = LayerNorm(config.n_embd, bias=config.bias)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = LayerNorm(config.n_embd, bias=config.bias)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

@dataclass
class Config:
    d_obs: int = 10
    d_act: int = 10
    block_size: int = 20
    n_layer: int = 4
    n_head: int = 4
    n_embd: int = 256

    dropout: float = 0
    bias: bool = True

class DecisionTransformer(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.block_size is not None
        self.config = config

        self.embed_rtg = nn.Linear(1, config.n_embd)
        self.embed_obs = nn.Linear(config.d_obs, config.n_embd)
        self.embed_act = nn.Linear(config.d_act, config.n_embd)
        self.wpe = nn.Embedding(config.block_size, config.n_embd)
        self.transformer = nn.ModuleDict(dict(
            drop = nn.Dropout(config.dropout),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = LayerNorm(config.n_embd, bias=config.bias),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.d_act, bias=False)
        # with weight tying when using torch.compile() some warnings get generated:
        # "UserWarning: functional_call was passed multiple values for tied weights.
        # This behavior is deprecated and will be an error in future versions"
        # not 100% sure what this is, so far seems to be harmless. TODO investigate
        # self.embed_act.weight = self.lm_head.weight # https://paperswithcode.com/method/weight-tying

        # init all weights
        self.apply(self._init_weights)
        # apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config.n_layer))

        # report number of parameters
        print("number of parameters: %.2fM" % (self.get_num_params()/1e6,))

    def get_num_params(self, non_embedding=True):
        """
        Return the number of parameters in the model.
        For non-embedding count (default), the position embeddings get subtracted.
        The token embeddings would too, except due to the parameter sharing these
        params are actually used as weights in the final layer, so we include them.
        """
        n_params = sum(p.numel() for p in self.parameters())
        if non_embedding:
            n_params -= self.wpe.weight.numel()
        return n_params

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, rtg, obs, act):
        device = rtg.device
        B, T, _ = obs.shape
        assert T <= self.config.block_size
        
        pos = torch.arange(0, T, dtype=torch.long, device=device) # shape (t)
        
        # forward the GPT model itself
        pos_emb = self.wpe(pos) # position embeddings of shape (t, n_embd)

        print(rtg.shape, obs.shape, act.shape)
        print(self.embed_rtg.weight.shape)
        print(self.embed_obs.weight.shape)
        print(self.embed_act.weight.shape)
        tok_rtg = pos_emb + self.embed_rtg(rtg) # token embeddings of shape (b, t, n_embd)
        tok_obs = pos_emb + self.embed_obs(obs) # token embeddings of shape (b, t, n_embd)
        tok_act = pos_emb + self.embed_act(act) # token embeddings of shape (b, t, n_embd)
        print('bef', tok_rtg.shape)
        x = rearrange([tok_rtg, tok_obs, tok_act], 'roa B T D -> B (T roa) D')
        print('aft', x.shape)
        
        x = self.transformer.drop(x)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        x = rearrange(x, 'B (T roa) D -> B T roa D', roa=3)
        print('a', x.shape)
        x = x[:, :, -1, :]
        print('b', x.shape)
        act_pred = self.lm_head(x)
        return act_pred

    def configure_optimizers(self, weight_decay, learning_rate, betas, device_type):
        # start with all of the candidate parameters
        param_dict = {pn: p for pn, p in self.named_parameters()}
        # filter out those that do not require grad
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
        print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == 'cuda'
        extra_args = dict(fused=True) if use_fused else dict()
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)
        print(f"using fused AdamW: {use_fused}")

        return optimizer

In [5]:
device = 'cuda:0'

In [6]:
config = Config(d_obs=32, d_act=8, block_size=20, n_layer=4, n_head=4, n_embd=256)
model = DecisionTransformer(config).to(device)

number of parameters: 3.17M


In [7]:
model(torch.randn(1, 20, 1).to(device), torch.randn(1, 20, 32).to(device), torch.randn(1, 20, 8).to(device))

torch.Size([1, 20, 1]) torch.Size([1, 20, 32]) torch.Size([1, 20, 8])
torch.Size([256, 1])
torch.Size([256, 32])
torch.Size([256, 8])
bef torch.Size([1, 20, 256])
aft torch.Size([1, 60, 256])
a torch.Size([1, 20, 3, 256])
b torch.Size([1, 20, 256])


tensor([[[ 2.0353e-01, -3.8569e-01,  3.9646e-01,  4.9481e-01,  6.7713e-01,
          -4.6159e-01,  7.9908e-01,  1.6425e-02],
         [-3.3877e-02,  1.4053e-01,  3.2075e-01,  3.9774e-01, -6.1029e-02,
           3.1290e-01, -6.1294e-02, -2.7331e-01],
         [-1.8243e-01,  1.6424e-01,  1.6112e-01, -3.1307e-01, -2.3705e-01,
          -9.5056e-02,  2.3738e-01, -5.6429e-02],
         [-1.9602e-01,  3.0718e-01,  1.3443e-03,  3.5842e-01, -3.1353e-01,
           3.1940e-01, -6.2291e-02, -3.8801e-01],
         [-6.5750e-02, -3.0030e-01, -1.2706e-02,  2.8188e-01, -1.2637e-01,
          -3.8196e-01,  3.9602e-01,  1.3794e-01],
         [ 9.0921e-01, -4.2611e-02, -1.4584e-01, -3.8703e-02, -3.9624e-01,
           3.0166e-01,  9.3948e-02, -4.2687e-01],
         [ 4.0376e-01, -8.7771e-02,  4.0322e-01, -2.7523e-01, -4.6761e-01,
          -4.3732e-02,  5.8071e-01, -4.2634e-01],
         [ 8.6821e-01,  4.8203e-01,  1.7311e-03, -7.8554e-02,  2.7821e-01,
           6.6845e-01,  2.4356e-01, -8.4900e-02],


In [2]:
import d4rl

/data/vision/phillipi/akumar01/.virtualenvs/dt/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:9: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn
/data/vision/phillipi/akumar01/.virtualenvs/dt/lib/python3.10/site-packages/mujoco_py/builder.py:9: DeprecationWarning: The distutils.sysconfig module is deprecated, use sysconfig instead
  from distutils.sysconfig import customize_compiler
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


Import error. Trying to rebuild mujoco_py.
Compiling /data/vision/phillipi/akumar01/.virtualenvs/dt/lib/python3.10/site-packages/mujoco_py/cymj.pyx because it changed.
[1/1] Cythonizing /data/vision/phillipi/akumar01/.virtualenvs/dt/lib/python3.10/site-packages/mujoco_py/cymj.pyx
running build_ext
building 'mujoco_py.cymj' extension
x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/data/vision/phillipi/akumar01/.virtualenvs/dt/lib/python3.10/site-packages/mujoco_py -I/data/vision/phillipi/akumar01/.mujoco/mujoco210/include -I/data/vision/phillipi/akumar01/.virtualenvs/dt/lib/python3.10/site-packages/numpy/core/include -I/data/vision/phillipi/akumar01/.virtualenvs/dt/include -I/usr/include/python3.10 -c /data/vision/phillipi/akumar01/.virtualenvs/dt/lib/python3.10/site-packages

<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
libglewosmesa.so: cannot open shared object file: No such file or directory
No module named 'flow'
No module named 'carla'
pybullet build time: Nov 28 2023 23:45:17


Import error. Trying to rebuild mujoco_py.
running build_ext
building 'mujoco_py.cymj' extension
x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/data/vision/phillipi/akumar01/.virtualenvs/dt/lib/python3.10/site-packages/mujoco_py -I/data/vision/phillipi/akumar01/.mujoco/mujoco210/include -I/data/vision/phillipi/akumar01/.virtualenvs/dt/lib/python3.10/site-packages/numpy/core/include -I/data/vision/phillipi/akumar01/.virtualenvs/dt/include -I/usr/include/python3.10 -c /data/vision/phillipi/akumar01/.virtualenvs/dt/lib/python3.10/site-packages/mujoco_py/cymj.c -o /data/vision/phillipi/akumar01/.virtualenvs/dt/lib/python3.10/site-packages/mujoco_py/generated/_pyxbld_2.1.2.14_310_linuxcpuextensionbuilder/temp.linux-x86_64-3.10/data/vision/phillipi/akumar01/.virtualenvs/dt/lib/py

<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/data/vision/phillipi/akumar01/.virtualenvs/dt/lib/python3.10/site-packages/mujoco_py -I/data/vision/phillipi/akumar01/.mujoco/mujoco210/include -I/data/vision/phillipi/akumar01/.virtualenvs/dt/lib/python3.10/site-packages/numpy/core/include -I/data/vision/phillipi/akumar01/.virtualenvs/dt/include -I/usr/include/python3.10 -c /data/vision/phillipi/akumar01/.virtualenvs/dt/lib/python3.10/site-packages/mujoco_py/gl/osmesashim.c -o /data/vision/phillipi/akumar01/.virtualenvs/dt/lib/python3.10/site-packages/mujoco_py/generated/_pyxbld_2.1.2.14_310_linuxcpuextensionbuilder/temp.linux-x86_64-3.10/data/vision/phillipi/akumar01/.virtualenvs/dt/lib/python3.10/site-packages/mujoco_py/gl/osmesashim.o -fopenmp -w
x86_64-linux-gnu-gcc -share

<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


DependencyNotInstalled: libglewosmesa.so: cannot open shared object file: No such file or directory. (HINT: you need to install mujoco_py, and also perform the setup instructions here: https://github.com/openai/mujoco-py/.)

In [ ]:
import gym